In [1]:
import pandas as pd
import numpy as np

In [2]:
#df = pd.read_json ('contacts.json')
#df.to_csv ('contacts.csv', index = None)

In [3]:
df = pd.read_csv('contacts.csv')
df.head()

,Id,Email,Phone,Contacts,OrderId
0,0,gkzAbIy@qq.com,NaN,1,NaN
1,1,NaN,3.294427e+11,4,vDDJJcxfLtSfkooPhbYnJdxov
2,2,NaN,9.125984e+09,0,NaN
3,3,mdllpYmE@gmail.com,NaN,0,bHquEnCbbsGLqllwryxPsNOxa
4,4,NaN,3.003644e+08,2,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Id        500000 non-null  int64  
 1   Email     348606 non-null  object 
 2   Phone     245081 non-null  float64
 3   Contacts  500000 non-null  int64  
 4   OrderId   243544 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 19.1+ MB


# Split Data Frame based on OrderId , Email and Phone

In [5]:
df_OrderId = df[df['OrderId'].notnull()]
print('df_OrderId : ',len(df_OrderId))

df_OrderId :  243544


In [6]:
df_Channel = df[df['OrderId'].isna()]
print('df_Channel : ',len(df_Channel))

df_Channel :  256456


In [7]:
df_Email = df_Channel[df_Channel['Email'].notnull()] # whether is phone null or not null is not a problem
print('df_Email : ',len(df_Email))

df_Email :  180646


In [8]:
df_Phone = df_Channel[df_Channel['Phone'].notnull() & df_Channel['Email'].isna()]
print('df_Phone : ',len(df_Phone))

df_Phone :  75810


# Process For OrderId Data Frame

In [9]:
df_OrderId['Contacts_OrderId'] = 0
df_OrderId['Ticket_Trace_OrderId'] = ''

<ipython-input-9-967b8f3e33ae>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_OrderId['Contacts_OrderId'] = 0
<ipython-input-9-967b8f3e33ae>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_OrderId['Ticket_Trace_OrderId'] = ''


## Calculate Contacts

In [10]:
sum_contacts = df_OrderId.groupby('OrderId')['Contacts'].sum()
df_sum_contacts = pd.DataFrame(sum_contacts.values, index=sum_contacts.index, columns=['Contacts_OrderId'])
df_sum_contacts = df_sum_contacts.reset_index()

df_OrderId = df_OrderId.merge(df_sum_contacts[['OrderId', 'Contacts_OrderId']], on=['OrderId'], how='left')
df_OrderId.drop('Contacts_OrderId_x', axis=1, inplace=True)
df_OrderId.rename({'Contacts_OrderId_y': 'Contacts_OrderId'}, axis=1, inplace=True)
df_OrderId

,Id,Email,Phone,Contacts,OrderId,Ticket_Trace_OrderId,Contacts_OrderId
0,1,NaN,3.294427e+11,4,vDDJJcxfLtSfkooPhbYnJdxov,,5
1,3,mdllpYmE@gmail.com,NaN,0,bHquEnCbbsGLqllwryxPsNOxa,,0
2,7,hqyunTiaXfIjhO@yahoo.com,NaN,1,vxjYZtMUvNhtEBDruzQfjpsmX,,1
3,9,osIIQgAiZX@hotmail.com,NaN,0,IqewrMXVTJqSsVklWFfOpdRNI,,1
4,10,NaN,9.189735e+11,3,OHRFAXqbAzyxljoUDaOErwtrg,,3
...,...,...,...,...,...,...,...
243539,499995,NaN,1.007240e+10,2,whJlUOGNqjxCRzmIRdURQvlNv,,2
243540,499996,NaN,5.717097e+11,4,JqIXOONvrwvJxZqNxCYHqnrKS,,4
243541,499997,NaN,4.541460e+09,2,beXCZSzcHaBwAYoDcpQqjuAFO,,2
243542,499998,RzSDsyH@hotmail.com,9.894719e+10,1,ehjeFACGiwrERQxbziMxwOWku,,5


## Combine Ticket Id

In [11]:
df_OrderId['Ticket_Trace_OrderId'] = df_OrderId['Id'].astype('str')
concat_id = df_OrderId.groupby(['OrderId'], as_index = False).agg({'Ticket_Trace_OrderId': '-'.join})

df_OrderId = df_OrderId.merge(concat_id[['OrderId', 'Ticket_Trace_OrderId']], on=['OrderId'], how='left') # like left join on SQL
df_OrderId.drop('Ticket_Trace_OrderId_x', axis=1, inplace=True)
df_OrderId.rename({'Ticket_Trace_OrderId_y': 'Ticket_Trace_OrderId'}, axis=1, inplace=True)
df_OrderId

,Id,Email,Phone,Contacts,OrderId,Contacts_OrderId,Ticket_Trace_OrderId
0,1,NaN,3.294427e+11,4,vDDJJcxfLtSfkooPhbYnJdxov,5,1-2458-476346
1,3,mdllpYmE@gmail.com,NaN,0,bHquEnCbbsGLqllwryxPsNOxa,0,3
2,7,hqyunTiaXfIjhO@yahoo.com,NaN,1,vxjYZtMUvNhtEBDruzQfjpsmX,1,7
3,9,osIIQgAiZX@hotmail.com,NaN,0,IqewrMXVTJqSsVklWFfOpdRNI,1,9-13-343161
4,10,NaN,9.189735e+11,3,OHRFAXqbAzyxljoUDaOErwtrg,3,10
...,...,...,...,...,...,...,...
243539,499995,NaN,1.007240e+10,2,whJlUOGNqjxCRzmIRdURQvlNv,2,499995
243540,499996,NaN,5.717097e+11,4,JqIXOONvrwvJxZqNxCYHqnrKS,4,499996
243541,499997,NaN,4.541460e+09,2,beXCZSzcHaBwAYoDcpQqjuAFO,2,499997
243542,499998,RzSDsyH@hotmail.com,9.894719e+10,1,ehjeFACGiwrERQxbziMxwOWku,5,121111-499998


## Check Final OrderId Data Frame

In [12]:
df_OrderId[df_OrderId['Ticket_Trace_OrderId'].apply(lambda x: len(x) > 40)]

,Id,Email,Phone,Contacts,OrderId,Contacts_OrderId,Ticket_Trace_OrderId
1656,3451,NaN,7.650693e+09,0,sXUrqwwNhKznjvToweHbZSEga,7,3451-19128-23681-47757-66674-122761-143699
9218,19128,YfPihfifRaECMEcxVC@gmail.com,9.916920e+11,0,sXUrqwwNhKznjvToweHbZSEga,7,3451-19128-23681-47757-66674-122761-143699
11395,23681,PfLYfCRVemFOWNEilv@gmail.com,2.280074e+10,4,sXUrqwwNhKznjvToweHbZSEga,7,3451-19128-23681-47757-66674-122761-143699
23119,47757,KbhZnppgQDNkTkkKX@yahoo.com,NaN,0,sXUrqwwNhKznjvToweHbZSEga,7,3451-19128-23681-47757-66674-122761-143699
32306,66674,FUhYsHRKrOFKFTkzOUv@yahoo.com,4.667519e+10,0,sXUrqwwNhKznjvToweHbZSEga,7,3451-19128-23681-47757-66674-122761-143699
47941,98509,NaN,1.048118e+11,0,XnkDQsOGUHYvPTGJPZQmkQUQr,10,98509-114513-131455-137736-220789-314641-457438
55757,114513,bPsEhke@yahoo.com,4.642194e+08,0,XnkDQsOGUHYvPTGJPZQmkQUQr,10,98509-114513-131455-137736-220789-314641-457438
59801,122761,tjXEinIgydcC@gmail.com,3.980891e+08,0,sXUrqwwNhKznjvToweHbZSEga,7,3451-19128-23681-47757-66674-122761-143699
62001,127242,NaN,8.831319e+09,1,ZdPDwjdbUWBzxgpJvbqtAPMqU,10,127242-143804-304832-391873-464525-494062
64080,131426,NaN,1.415482e+08,0,zxifcdEzTObwIFwGCyPQgWBRI,8,131426-141956-145882-239291-279311-405666


# Process For Email Data Frame

In [13]:
df_Email['Contacts_Email'] = 0
df_Email['Ticket_Trace_Email'] = ''

<ipython-input-13-a0254d1be279>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Email['Contacts_Email'] = 0
<ipython-input-13-a0254d1be279>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Email['Ticket_Trace_Email'] = ''


## Calculate Contacts

In [14]:
sum_contacts = df_Email.groupby('Email')['Contacts'].sum()
df_sum_contacts = pd.DataFrame(sum_contacts.values, index=sum_contacts.index, columns=['Contacts_Email'])
df_sum_contacts = df_sum_contacts.reset_index()

df_Email = df_Email.merge(df_sum_contacts[['Email', 'Contacts_Email']], on=['Email'], how='left')
df_Email.drop('Contacts_Email_x', axis=1, inplace=True)
df_Email.rename({'Contacts_Email_y': 'Contacts_Email'}, axis=1, inplace=True)
df_Email

,Id,Email,Phone,Contacts,OrderId,Ticket_Trace_Email,Contacts_Email
0,0,gkzAbIy@qq.com,NaN,1,NaN,,1
1,6,hESiBgYTxMnj@hotmail.com,NaN,4,NaN,,4
2,8,XGUjcSiDpFKWEUCs@qq.com,2.236046e+11,2,NaN,,2
3,11,QRamvop@qq.com,NaN,0,NaN,,3
4,12,QexYMLXc@qq.com,NaN,2,NaN,,4
...,...,...,...,...,...,...,...
180641,499972,uHawkwWbcMKjjdv@qq.com,NaN,0,NaN,,0
180642,499981,eNWjHmCSJvl@qq.com,NaN,3,NaN,,3
180643,499984,EPbtDNyUjJmARoOl@qq.com,NaN,3,NaN,,3
180644,499986,zxSchUrACg@gmail.com,2.016079e+07,1,NaN,,1


## Combine Ticket Id

In [15]:
df_Email['Ticket_Trace_Email'] = df_Email['Id'].astype('str')
concat_id = df_Email.groupby(['Email'], as_index = False).agg({'Ticket_Trace_Email': '-'.join})

df_Email = df_Email.merge(concat_id[['Email', 'Ticket_Trace_Email']], on=['Email'], how='left') # like left join on SQL
df_Email.drop('Ticket_Trace_Email_x', axis=1, inplace=True)
df_Email.rename({'Ticket_Trace_Email_y': 'Ticket_Trace_Email'}, axis=1, inplace=True)
df_Email

,Id,Email,Phone,Contacts,OrderId,Contacts_Email,Ticket_Trace_Email
0,0,gkzAbIy@qq.com,NaN,1,NaN,1,0
1,6,hESiBgYTxMnj@hotmail.com,NaN,4,NaN,4,6-38
2,8,XGUjcSiDpFKWEUCs@qq.com,2.236046e+11,2,NaN,2,8
3,11,QRamvop@qq.com,NaN,0,NaN,3,11-244207
4,12,QexYMLXc@qq.com,NaN,2,NaN,4,12-480595
...,...,...,...,...,...,...,...
180641,499972,uHawkwWbcMKjjdv@qq.com,NaN,0,NaN,0,499972
180642,499981,eNWjHmCSJvl@qq.com,NaN,3,NaN,3,499981
180643,499984,EPbtDNyUjJmARoOl@qq.com,NaN,3,NaN,3,499984
180644,499986,zxSchUrACg@gmail.com,2.016079e+07,1,NaN,1,499986


## Check Final Email Data Frame

In [16]:
df_Email[df_Email['Ticket_Trace_Email'].apply(lambda x: len(x) > 30)]

,Id,Email,Phone,Contacts,OrderId,Contacts_Email,Ticket_Trace_Email
2018,5524,VfOMHMVq@yahoo.com,NaN,2,NaN,6,5524-108220-293348-341368-441156
3647,10096,TDouXgBeZec@hotmail.com,NaN,3,NaN,13,10096-85211-147486-212735-400760
3922,10830,cVXvNxukDft@qq.com,NaN,1,NaN,6,10830-38232-55125-331745-470000
4686,12887,AhshBfIGkVN@gmail.com,8.222494e+09,4,NaN,11,12887-352743-356468-399780-401581
5766,15814,joeiBlUUXIEsEBHp@yahoo.com,NaN,2,NaN,11,15814-36831-119674-148588-292880
...,...,...,...,...,...,...,...
175198,484971,XdRNyNzOquPmQGLVJgh@yahoo.com,NaN,4,NaN,11,269370-269527-352509-354930-484971
175265,485176,GUacPhNRmsYZPtS@qq.com,8.248531e+10,1,NaN,11,135059-143882-423771-471423-485176
175838,486795,HvyrzFHpAxToZvx@qq.com,1.644369e+08,4,NaN,10,108978-149586-161657-406650-486795
176819,489536,DsArmrwnjTgIAcqu@qq.com,8.623143e+08,2,NaN,8,39376-105165-216287-292047-489536


# Process For Phone Data Frame

In [17]:
df_Phone['Contacts_Phone'] = 0
df_Phone['Ticket_Trace_Phone'] = ''

<ipython-input-17-29d7a5aa7d65>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Phone['Contacts_Phone'] = 0
<ipython-input-17-29d7a5aa7d65>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Phone['Ticket_Trace_Phone'] = ''


## Calculate Contacts

In [18]:
sum_contacts = df_Phone.groupby('Phone')['Contacts'].sum()
df_sum_contacts = pd.DataFrame(sum_contacts.values, index=sum_contacts.index, columns=['Contacts_Phone'])
df_sum_contacts = df_sum_contacts.reset_index()

df_Phone = df_Phone.merge(df_sum_contacts[['Phone', 'Contacts_Phone']], on=['Phone'], how='left')
df_Phone.drop('Contacts_Phone_x', axis=1, inplace=True)
df_Phone.rename({'Contacts_Phone_y': 'Contacts_Phone'}, axis=1, inplace=True)
df_Phone

,Id,Email,Phone,Contacts,OrderId,Ticket_Trace_Phone,Contacts_Phone
0,2,NaN,9.125984e+09,0,NaN,,0
1,4,NaN,3.003644e+08,2,NaN,,2
2,5,NaN,8.401131e+08,0,NaN,,0
3,14,NaN,1.742380e+10,4,NaN,,4
4,15,NaN,9.260790e+10,4,NaN,,4
...,...,...,...,...,...,...,...
75805,499976,NaN,9.759965e+10,1,NaN,,1
75806,499977,NaN,2.570199e+08,3,NaN,,3
75807,499979,NaN,7.514380e+11,2,NaN,,2
75808,499982,NaN,5.232281e+08,3,NaN,,3


## Combine Ticket Id

In [19]:
df_Phone['Ticket_Trace_Phone'] = df_Phone['Id'].astype('str')
concat_id = df_Phone.groupby(['Phone'], as_index = False).agg({'Ticket_Trace_Phone': '-'.join})

df_Phone = df_Phone.merge(concat_id[['Phone', 'Ticket_Trace_Phone']], on=['Phone'], how='left') # like left join on SQL
df_Phone.drop('Ticket_Trace_Phone_x', axis=1, inplace=True)
df_Phone.rename({'Ticket_Trace_Phone_y': 'Ticket_Trace_Phone'}, axis=1, inplace=True)
df_Phone

,Id,Email,Phone,Contacts,OrderId,Contacts_Phone,Ticket_Trace_Phone
0,2,NaN,9.125984e+09,0,NaN,0,2
1,4,NaN,3.003644e+08,2,NaN,2,4
2,5,NaN,8.401131e+08,0,NaN,0,5
3,14,NaN,1.742380e+10,4,NaN,4,14
4,15,NaN,9.260790e+10,4,NaN,4,15
...,...,...,...,...,...,...,...
75805,499976,NaN,9.759965e+10,1,NaN,1,499976
75806,499977,NaN,2.570199e+08,3,NaN,3,376850-499977
75807,499979,NaN,7.514380e+11,2,NaN,2,499979
75808,499982,NaN,5.232281e+08,3,NaN,3,499982


## Check Final Phone Data Frame

In [20]:
df_Phone[df_Phone['Ticket_Trace_Phone'].apply(lambda x: len(x) > 20)]

,Id,Email,Phone,Contacts,OrderId,Contacts_Phone,Ticket_Trace_Phone
1413,9168,NaN,7.995227e+10,3,NaN,7,9168-118899-295093-360203
2013,13147,NaN,5.380510e+08,0,NaN,12,13147-148633-184203-493968
2148,13964,NaN,4.963097e+09,2,NaN,10,13964-211582-379189-498558
5336,35283,NaN,5.762336e+09,1,NaN,5,35283-144941-363352-446185
17915,118899,NaN,7.995227e+10,0,NaN,7,9168-118899-295093-360203
18808,125011,NaN,9.748833e+11,1,NaN,8,125011-135424-262303-370715
20363,135424,NaN,9.748833e+11,3,NaN,8,125011-135424-262303-370715
21845,144941,NaN,5.762336e+09,2,NaN,5,35283-144941-363352-446185
22403,148633,NaN,5.380510e+08,4,NaN,12,13147-148633-184203-493968
24627,163222,NaN,8.160915e+11,4,NaN,13,163222-173321-240024-311178


# Find Intersection between OrderId and Email Data Frame based on Email

In [21]:
df_Intersec_OrderEmail = df_OrderId.merge(df_Email[['Email', 'Contacts_Email','Ticket_Trace_Email']], on=['Email'], how='inner') # like inner join on SQL
df_Intersec_OrderEmail

,Id,Email,Phone,Contacts,OrderId,Contacts_OrderId,Ticket_Trace_OrderId,Contacts_Email,Ticket_Trace_Email
0,19,OPDjDybln@qq.com,NaN,1,hmEXuNUPZQyefHErmzjGbeeFK,1,19,0,406780
1,21,BWoPWiRuMzWyInsDy@gmail.com,NaN,1,MswKjQqGNxFUOgZyDtiksGKhJ,1,21,4,499255
2,30,mgKGRKoFOdsB@yahoo.com,NaN,1,krTjhUVfqiDZCGFaQpJpQlYBs,1,30,3,19563
3,42,VbaKKxkCzo@yahoo.com,3.151944e+08,3,TsBwFmhSCeauMjfoMVkhTBdYI,3,42,3,164913
4,52,ITSENNNBYRhq@yahoo.com,NaN,2,JsORSKHmzWlRvSNKUTVofoPkZ,2,52-340482,4,394326
...,...,...,...,...,...,...,...,...,...
62597,499928,jnvukHqSFtXBHqYiqB@hotmail.com,NaN,4,hAMoOJoATqTLSFzIQmRXoWxdu,8,408266-499928,4,368836
62598,499931,FIDpvjmFaibqFj@hotmail.com,NaN,2,QvVvdmDXEvShHdTmmaQUODuZF,3,489541-499931,4,49663-251252
62599,499931,FIDpvjmFaibqFj@hotmail.com,NaN,2,QvVvdmDXEvShHdTmmaQUODuZF,3,489541-499931,4,49663-251252
62600,499944,OfuKcOBdzWtPiQOxRYiu@hotmail.com,6.507453e+11,3,sEeNiHgvXEAOSFFgcPDIFDunU,3,171336-499944,2,224605


In [22]:
df_Intersec_OrderEmail['Contacts_IOE'] = df_Intersec_OrderEmail['Contacts_OrderId'] + df_Intersec_OrderEmail['Contacts_Email']
df_Intersec_OrderEmail['Ticket_Trace_IOE'] = df_Intersec_OrderEmail[['Ticket_Trace_OrderId', 'Ticket_Trace_Email']].agg('-'.join, axis=1)
df_Intersec_OrderEmail.drop(['Phone', 'Contacts', 'OrderId', 'Contacts_OrderId','Ticket_Trace_OrderId', 'Contacts_Email','Ticket_Trace_Email'], axis=1, inplace=True)
df_Intersec_OrderEmail

,Id,Email,Contacts_IOE,Ticket_Trace_IOE
0,19,OPDjDybln@qq.com,1,19-406780
1,21,BWoPWiRuMzWyInsDy@gmail.com,5,21-499255
2,30,mgKGRKoFOdsB@yahoo.com,4,30-19563
3,42,VbaKKxkCzo@yahoo.com,6,42-164913
4,52,ITSENNNBYRhq@yahoo.com,6,52-340482-394326
...,...,...,...,...
62597,499928,jnvukHqSFtXBHqYiqB@hotmail.com,12,408266-499928-368836
62598,499931,FIDpvjmFaibqFj@hotmail.com,7,489541-499931-49663-251252
62599,499931,FIDpvjmFaibqFj@hotmail.com,7,489541-499931-49663-251252
62600,499944,OfuKcOBdzWtPiQOxRYiu@hotmail.com,5,171336-499944-224605


# Find Intersection between OrderId and Phone Data Frame based on Phone

In [23]:
df_Intersec_OrderPhone = df_OrderId.merge(df_Phone[['Phone', 'Contacts_Phone','Ticket_Trace_Phone']], on=['Phone'], how='inner') # like inner join on SQL
df_Intersec_OrderPhone

,Id,Email,Phone,Contacts,OrderId,Contacts_OrderId,Ticket_Trace_OrderId,Contacts_Phone,Ticket_Trace_Phone
0,17,TNaMorNIWCJARsXlg@gmail.com,9.260790e+10,3,GqpDeqZSwZTbPRrrGePkOhbSQ,3,17,4,15
1,26,NaN,3.870650e+08,0,lIuGVCoYTcpMcQWyGusHFwJNF,1,26-402598,1,229592
2,50,TIMejlVYhfqjy@gmail.com,8.401131e+08,4,XBkNrwcWWslDvSQdhYPfCpQJo,8,50-226720-458692,0,5
3,51,LEQvQVjRnZGUzzii@qq.com,6.770717e+08,4,tNyZAarWISkKcyhLPjYOVXfrD,4,51,1,35
4,68,MqeWqLfAIPvgaelFet@hotmail.com,1.737545e+11,3,yszJKOtWYrnjJoMPmXQMuZhiu,3,68,2,1321
...,...,...,...,...,...,...,...,...,...
18298,499643,NaN,5.256755e+10,4,bWSAKSlfSVDDCjhRszoAskpjD,4,499643,4,262579
18299,499711,ldkUonUOvrjC@gmail.com,2.927701e+09,2,GguITazcDQfoPMQsUEDaXVoSU,2,499711,1,149700
18300,499800,NaN,1.992171e+10,3,DCRpccIEshwtcqJtfsyJHSUmo,3,499800,3,210893
18301,499881,NaN,1.262341e+11,3,EcnbxEnUteqTsKvsyKMiKgQFT,9,239767-376391-499881,1,453367


In [24]:
df_Intersec_OrderPhone['Contacts_IOP'] = df_Intersec_OrderPhone['Contacts_OrderId'] + df_Intersec_OrderPhone['Contacts_Phone']
df_Intersec_OrderPhone['Ticket_Trace_IOP'] = df_Intersec_OrderPhone[['Ticket_Trace_OrderId', 'Ticket_Trace_Phone']].agg('-'.join, axis=1)
df_Intersec_OrderPhone.drop(['Email', 'Contacts', 'OrderId', 'Contacts_OrderId','Ticket_Trace_OrderId', 'Contacts_Phone','Ticket_Trace_Phone'], axis=1, inplace=True)
df_Intersec_OrderPhone

,Id,Phone,Contacts_IOP,Ticket_Trace_IOP
0,17,9.260790e+10,7,17-15
1,26,3.870650e+08,2,26-402598-229592
2,50,8.401131e+08,8,50-226720-458692-5
3,51,6.770717e+08,5,51-35
4,68,1.737545e+11,5,68-1321
...,...,...,...,...
18298,499643,5.256755e+10,8,499643-262579
18299,499711,2.927701e+09,3,499711-149700
18300,499800,1.992171e+10,6,499800-210893
18301,499881,1.262341e+11,10,239767-376391-499881-453367


# Merge Intersection OrderEmail

## Merge To OrderId

In [30]:
len(df_OrderId)

243544

In [37]:
len(df_Intersec_OrderEmail)

62602

In [38]:
tt = df_OrderId.merge(df_Intersec_OrderEmail[['Email','Contacts_IOE','Ticket_Trace_IOE']], on=['Email'], how='left') # like inner join on SQL
tt

,Id,Email,Phone,Contacts,OrderId,Contacts_OrderId,Ticket_Trace_OrderId,Contacts_IOE,Ticket_Trace_IOE
0,1,NaN,3.294427e+11,4,vDDJJcxfLtSfkooPhbYnJdxov,5,1-2458-476346,NaN,NaN
1,3,mdllpYmE@gmail.com,NaN,0,bHquEnCbbsGLqllwryxPsNOxa,0,3,NaN,NaN
2,7,hqyunTiaXfIjhO@yahoo.com,NaN,1,vxjYZtMUvNhtEBDruzQfjpsmX,1,7,NaN,NaN
3,9,osIIQgAiZX@hotmail.com,NaN,0,IqewrMXVTJqSsVklWFfOpdRNI,1,9-13-343161,NaN,NaN
4,10,NaN,9.189735e+11,3,OHRFAXqbAzyxljoUDaOErwtrg,3,10,NaN,NaN
...,...,...,...,...,...,...,...,...,...
276805,499995,NaN,1.007240e+10,2,whJlUOGNqjxCRzmIRdURQvlNv,2,499995,NaN,NaN
276806,499996,NaN,5.717097e+11,4,JqIXOONvrwvJxZqNxCYHqnrKS,4,499996,NaN,NaN
276807,499997,NaN,4.541460e+09,2,beXCZSzcHaBwAYoDcpQqjuAFO,2,499997,NaN,NaN
276808,499998,RzSDsyH@hotmail.com,9.894719e+10,1,ehjeFACGiwrERQxbziMxwOWku,5,121111-499998,NaN,NaN


## Merge To Email

In [25]:
##############

In [26]:
y = df_OrderId.merge(df_Intersec_OrderEmail[['Email', 'Contacts_IOE', 'Ticket_Trace_IOE']], on=['Email'], how='left') # like left join on SQL
y

,Id,Email,Phone,Contacts,OrderId,Contacts_OrderId,Ticket_Trace_OrderId,Contacts_IOE,Ticket_Trace_IOE
0,1,NaN,3.294427e+11,4,vDDJJcxfLtSfkooPhbYnJdxov,5,1-2458-476346,NaN,NaN
1,3,mdllpYmE@gmail.com,NaN,0,bHquEnCbbsGLqllwryxPsNOxa,0,3,NaN,NaN
2,7,hqyunTiaXfIjhO@yahoo.com,NaN,1,vxjYZtMUvNhtEBDruzQfjpsmX,1,7,NaN,NaN
3,9,osIIQgAiZX@hotmail.com,NaN,0,IqewrMXVTJqSsVklWFfOpdRNI,1,9-13-343161,NaN,NaN
4,10,NaN,9.189735e+11,3,OHRFAXqbAzyxljoUDaOErwtrg,3,10,NaN,NaN
...,...,...,...,...,...,...,...,...,...
276805,499995,NaN,1.007240e+10,2,whJlUOGNqjxCRzmIRdURQvlNv,2,499995,NaN,NaN
276806,499996,NaN,5.717097e+11,4,JqIXOONvrwvJxZqNxCYHqnrKS,4,499996,NaN,NaN
276807,499997,NaN,4.541460e+09,2,beXCZSzcHaBwAYoDcpQqjuAFO,2,499997,NaN,NaN
276808,499998,RzSDsyH@hotmail.com,9.894719e+10,1,ehjeFACGiwrERQxbziMxwOWku,5,121111-499998,NaN,NaN


In [27]:
x = y.merge(df_Intersec_OrderPhone[['Phone', 'Contacts_IOP', 'Ticket_Trace_IOP']], on=['Phone'], how='left') # like left join on SQL
x

,Id,Email,Phone,Contacts,OrderId,Contacts_OrderId,Ticket_Trace_OrderId,Contacts_IOE,Ticket_Trace_IOE,Contacts_IOP,Ticket_Trace_IOP
0,1,NaN,3.294427e+11,4,vDDJJcxfLtSfkooPhbYnJdxov,5,1-2458-476346,NaN,NaN,NaN,NaN
1,3,mdllpYmE@gmail.com,NaN,0,bHquEnCbbsGLqllwryxPsNOxa,0,3,NaN,NaN,NaN,NaN
2,7,hqyunTiaXfIjhO@yahoo.com,NaN,1,vxjYZtMUvNhtEBDruzQfjpsmX,1,7,NaN,NaN,NaN,NaN
3,9,osIIQgAiZX@hotmail.com,NaN,0,IqewrMXVTJqSsVklWFfOpdRNI,1,9-13-343161,NaN,NaN,NaN,NaN
4,10,NaN,9.189735e+11,3,OHRFAXqbAzyxljoUDaOErwtrg,3,10,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
283521,499995,NaN,1.007240e+10,2,whJlUOGNqjxCRzmIRdURQvlNv,2,499995,NaN,NaN,NaN,NaN
283522,499996,NaN,5.717097e+11,4,JqIXOONvrwvJxZqNxCYHqnrKS,4,499996,NaN,NaN,NaN,NaN
283523,499997,NaN,4.541460e+09,2,beXCZSzcHaBwAYoDcpQqjuAFO,2,499997,NaN,NaN,NaN,NaN
283524,499998,RzSDsyH@hotmail.com,9.894719e+10,1,ehjeFACGiwrERQxbziMxwOWku,5,121111-499998,NaN,NaN,NaN,NaN
